In [ ]:
import os
import random
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

# added the packages that are needed for THE ASSIGNMENT HERE

In [ ]:
# Configuration parameters
# to facilitate the code i added these variables so we can change them easily
DATA_DIR = "/Users/bryangutierrez/Desktop/project3_ML"   
BATCH_SIZE = 8
NUM_EPOCHS = 50
LEARNING_RATE = 1e-4
VAL_SPLIT = 0.3
RANDOM_SEED = 43
 
MODEL_PATH = "Group_16_CNN_FullModel.ph"

# THis to make the code reproducible
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# this is to detect whether your computer has a GPU which is Cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
# Data transformations
# this prepares the images so the cnn can process them 
# rezize, normalize, augment the data with random horizontal flip
train_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),

    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# Validation transformations
# this prepares the images so the cnn can process them 
val_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])



In [ ]:
# Load the full dataset
full_dataset = datasets.ImageFolder(root=DATA_DIR, transform=train_transform)

class_names = full_dataset.classes
print("Classes:", class_names)  

# Train/val split
# this splits the data into training and validation sets
dataset_size = len(full_dataset)
val_size = int(VAL_SPLIT * dataset_size)
train_size = dataset_size - val_size
# this makes the split reproducible by setting a manual seed
train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(RANDOM_SEED)
)

# Update transforms for validation dataset
# this applies the validation transformations to the validation dataset
val_dataset.dataset.transform = val_transform
# Data loaders which handle batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Total images: {dataset_size}")
print(f"Train size: {train_size}, Val size: {val_size}")



Classes: ['not_official', 'official']
Total images: 50
Train size: 35, Val size: 15


In [ ]:
# Define the CNN model
# This is a custom CNN architecture named RazorbackCNN
class RazorbackCNN(nn.Module):


   # this is the constructor of the class
    def __init__(self, num_classes=2):
        super(RazorbackCNN, self).__init__()

# this defines the layers of the CNN

        self.features = nn.Sequential(
            # Block 1


            # this block processes the input image
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 500 -> 250

            # Block 2
            # this block extracts more complex features
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 250 -> 125

            # Block 3
            # this block further refines the features
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 125 -> 62/63

            # Block 4
            # this block captures high-level features
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # ~62 -> ~31

           
            nn.AdaptiveAvgPool2d((4, 4))
        )
# this defines the fully connected layers for classification
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # 2 classes
        )
# this defines the forward pass of the model
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = RazorbackCNN(num_classes=2).to(device) # move the model to the appropriate device
print(model)



RazorbackCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): AdaptiveAvgPool2d(output_size=(4, 4))
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=256, bias=True)
    (2): ReLU(inplace=Tru

In [ ]:
# Loss function and optimizer
# this defines the loss function and optimizer for training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)



In [ ]:
# Training and evaluation functions
def train_one_epoch(model, dataloader, criterion, optimizer, device): # this function trains the model for one epoch
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0


# this loops over the data in the dataloader
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # reset gradients

        outputs = model(images)
        loss = criterion(outputs, labels)


# computes the predictions and updates the loss and accuracy
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()
# updates the running loss and correct predictions
        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels).item()
        total += labels.size(0)


# computes the epoch loss and accuracy
    epoch_loss = running_loss / total
    epoch_acc = running_corrects / total
    return epoch_loss, epoch_acc

# this function evaluates the model on the validation set
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0




#  loops over the data in the dataloader without computing gradients
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images) # get model outputs
            loss = criterion(outputs, labels) # computes the loss

            _, preds = torch.max(outputs, 1) # computes the predictions
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels).item()
            total += labels.size(0)


# THUS computes the epoch loss and accuracy
    epoch_loss = running_loss / total
    epoch_acc = running_corrects / total
    return epoch_loss, epoch_acc



In [ ]:
# Training loop
# this is the main training loop that runs for epochs
best_val_acc = 0.0
best_model_state = None

# this loops over the number of epochs
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device) # train for one epoch
    val_loss, val_acc = evaluate(model, val_loader, criterion, device) # evaluate on validation set

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
          f"|| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Save best model
    # this saves the model if it has the best validation accuracy so far
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict()

print(f"\nBest validation accuracy: {best_val_acc:.4f}")

# Load best model state
# this loads the best model state after training
if best_model_state is not None:
    model.load_state_dict(best_model_state)



Epoch [1/50] Train Loss: 0.6859 | Train Acc: 0.6286 || Val Loss: 0.6722 | Val Acc: 0.6667
Epoch [2/50] Train Loss: 0.6617 | Train Acc: 0.7143 || Val Loss: 0.6425 | Val Acc: 0.6667
Epoch [3/50] Train Loss: 0.6277 | Train Acc: 0.7143 || Val Loss: 0.6196 | Val Acc: 0.6667
Epoch [4/50] Train Loss: 0.6028 | Train Acc: 0.7143 || Val Loss: 0.6132 | Val Acc: 0.6667
Epoch [5/50] Train Loss: 0.5715 | Train Acc: 0.7143 || Val Loss: 0.6100 | Val Acc: 0.6667
Epoch [6/50] Train Loss: 0.5731 | Train Acc: 0.7143 || Val Loss: 0.6151 | Val Acc: 0.6667
Epoch [7/50] Train Loss: 0.5789 | Train Acc: 0.7143 || Val Loss: 0.6193 | Val Acc: 0.6667
Epoch [8/50] Train Loss: 0.5745 | Train Acc: 0.7143 || Val Loss: 0.6214 | Val Acc: 0.6667
Epoch [9/50] Train Loss: 0.5484 | Train Acc: 0.7143 || Val Loss: 0.6211 | Val Acc: 0.6667
Epoch [10/50] Train Loss: 0.5746 | Train Acc: 0.7143 || Val Loss: 0.6161 | Val Acc: 0.6667
Epoch [11/50] Train Loss: 0.5607 | Train Acc: 0.7143 || Val Loss: 0.6128 | Val Acc: 0.6667
Epoch [1

In [36]:
# Save the full model
torch.save(model, MODEL_PATH)
print(f"Saved full model to: {MODEL_PATH}")


Saved full model to: Group_16_CNN_FullModel.ph
